# Measuring inductance with signal generator and Oscilloscope

In [1]:
import pyvisa
import time
import math

In [2]:
Freq = 1000
step = 500

In [3]:
rm = pyvisa.ResourceManager()

In [4]:
print(rm.list_resources())

('USB0::0x1AB1::0x04CE::DS1ZA222706984::INSTR', 'USB0::0x1AB1::0x0642::DG1ZA000000000::INSTR', 'USB0::0x0483::0x7540::AMM2524010088::INSTR', 'ASRL4::INSTR', 'ASRL6::INSTR', 'ASRL11::INSTR', 'ASRL13::INSTR', 'ASRL14::INSTR')


In [5]:
ut8805 = rm.open_resource('USB0::0x0483::0x7540::AMM2524010088::INSTR')
dg1022 = rm.open_resource('USB0::0x1AB1::0x0642::DG1ZA000000000::INSTR')
ds1054 = rm.open_resource('USB0::0x1AB1::0x04CE::DS1ZA222706984::INSTR')

## Connect a 180 Ohm resistor to the multimeter

In [ ]:
input('Press enter to continue....')
ut8805.write('CONF:RES')
resistance = float(ut8805.query('MEAS:RES?'))
print (resistance)

177.819427


#### Setting signal generator output to 1 Khz sine, 3 v p-p

In [7]:
dg1022.write(':SOUR1:APPL:SIN '+ str(Freq) +',3,0')
dg1022.write(':OUTP1 ON')

11

#### Setting Oscilloscope parameters

In [8]:
ds1054.write(':TIM:MAIN:SCAL 0.0002')
ds1054.write(':CHAN1:SCAL 0.5')
time.sleep(5)

In [9]:
Mvpp1 = float(ds1054.query(':MEAS:ITEM? VPP, CHAN1'))
print(Mvpp1)

3.04


In [10]:
print('Connect resistor to output of signal generator')
input("Press enter to continue..")

Connect resistor to output of signal generator


Press enter to continue.. 


''

In [11]:
Mvpp2 = float(ds1054.query(':MEAS:ITEM? VPP, CHAN1'))
print(Mvpp2)

2.38


In [12]:
OutImp = float(resistance*((Mvpp1/Mvpp2)-1))
print (OutImp)

49.31126967226893


#### Remove the resister from the funtion generator and connect an inductor

In [13]:
input("Press enter to continue...")

Press enter to continue... 


''

In [14]:
Freq = 20000
dg1022.write (':SOUR1:APPL:SIN '+str(Freq)+',3,0')
time.sleep(5)
ds1054.write(':TIM:MAIN:SCAL 0.000005')

25

In [15]:
Mvpp3 = float(ds1054.query(':MEAS:ITEM? VPP, CHAN1'))
print(Mvpp3)

0.42


In [16]:
if Mvpp3 > 0.5*Mvpp1:
    step = 500
    while Mvpp3 >= 0.5*Mvpp1:
        Freq -= step
        dg1022.write(':SOUR1:APPL:SiN '+str(Freq)+',3,0')
        time.sleep(1)
        Mvpp3 = float(ds1054.query(':MEAS:ITEM? VPP,CHAN1'))
else:
    while Mvpp3 <= 0.5*Mvpp1:
        step = ((0.5*Mvpp1)-Mvpp3)*10000
        Freq += step
        dg1022.write(':SOUR1:APPL:SiN '+str(Freq)+',3,0')
        time.sleep(1)
        Mvpp3 = float(ds1054.query(':MEAS:ITEM? VPP,CHAN1'))

In [17]:
f = dg1022.query(':SOUR1:FREQ?')
Ind = (1e6*(math.sqrt(1/3)*(OutImp/(2*math.pi * float(f)))))
print("%.3g" %Ind+"uH")

47.8uH
